<a href="https://colab.research.google.com/github/everluiz/Machine-learning-class/blob/main/model_selection_Formula1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import time
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/f1.csv')

df.head()

,id,Time-Q1,Time-Q2,Time-Q3,driver,Pos,Year,Races,Fastest Lap,Grid,Team
0,1,1:25.900,1:25.347,1:24.303,Lewis Hamilton,1.0,2020,Silverstone,1:28.689,1,Mercedes
1,2,1:25.801,1:25.015,1:24.616,Valtteri Bottas,11.0,2020,Silverstone,1:27.097,2,Mercedes
2,3,1:25.513,1:25.840,1:25.099,Lewis Hamilton,1.0,2019,Silverstone,1:27.369,2,Mercedes
3,4,1:25.750,1:25.672,1:25.093,Valtteri Bottas,2.0,2019,Silverstone,1:27.406,1,Mercedes
4,5,1:30.739,1:29.243,1:29.287,Lewis Hamilton,1.0,2016,Silverstone,1:35.771,1,Mercedes


Mapeamento dos nomes das equipes de corrida (classes)

e mapeando os nomes das caracteristicas pilotos e pistas

Eliminação das amostras com valores faltando

In [ ]:
T = {'Mercedes': 0,
     'Red Bull': 1,}

d = {'Lewis Hamilton': 0,
     'Valtteri Bottas': 1,
     'Nico Rosberg': 2,
     'Max Verstappen': 3,
     'Alexander Albon': 4,
     'Pierre Gasly': 5,
     'Daniel Ricciardo': 6}

R = {'Silverstone': 0,
     'Austria': 1,
     'Barcelona': 2,
     'Hungria': 3}
     

df['Team'] = df['Team'].map(T)
df['driver'] = df['driver'].map(d)
df['Races'] = df['Races'].map(R)
df = df.dropna() # Elimina linhas com NaN (valores faltando)
df.head()

,id,Time-Q1,Time-Q2,Time-Q3,driver,Pos,Year,Races,Fastest Lap,Grid,Team
0,1,1:25.900,1:25.347,1:24.303,0.0,1.0,2020,0,1:28.689,1,0
2,3,1:25.513,1:25.840,1:25.099,0.0,1.0,2019,0,1:27.369,2,0
4,5,1:30.739,1:29.243,1:29.287,0.0,1.0,2016,0,1:35.771,1,0
5,6,1:30.724,1:29.970,1:29.606,2.0,3.0,2016,0,1:35.548,2,0
6,7,1:04.198,1:03.096,1:02.951,0.0,4.0,2020,1,1:07.712,2,0


Matriz X com valores das caracteristicas

Vetor Y com os valores das classes

loop for para converter strings do tempo dos pilotos para timestamp (float)

In [ ]:
X = df.iloc[:, 1:10].values
y = df['Team'].values

for i in range(0,len(X[:,])):
    dt = datetime.strptime(X[i,0], "%M:%S.%f")
    X[i,0] = time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0) # conversão para timestamp
    dt = datetime.strptime(X[i,1], "%M:%S.%f")
    X[i,1] = time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0) # conversão para timestamp
    dt = datetime.strptime(X[i,2], "%M:%S.%f")
    X[i,2] = time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0) # conversão para timestamp
    dt = datetime.strptime(X[i,7], "%M:%S.%f")
    X[i,7] = time.mktime(dt.timetuple()) + (dt.microsecond / 1000000.0) # conversão para timestamp
    #print(datetime.utcfromtimestamp(X[4,0]).strftime('%M:%S.%f')) # print do tempo

dividindo os dados para treino e teste, com 20% das amostras para teste (80% para treino), embaralhando os dados randômicamente e estratificando os grupos de teste e treinamento homogeneamente.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=123, stratify=y)

Criando objeto Pipeline:


*   standardScaler (normalização com media 0 e desvio padrão 1)
*   PCA (redução de dimensionalidade "3 dimensões")
*   classificador KNN




In [ ]:
pipe = Pipeline([
        ('z-score', StandardScaler()),
        ('reduce_dim', PCA(n_components=3)),
        ('classify', KNeighborsClassifier(n_neighbors=1))])

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('z-score',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('classify',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=1, p=2,
                                      weights='uniform'))],
         verbose=False)

Teste de acurácia com base de treino

In [ ]:
from sklearn.metrics import accuracy_score

y_train_pred = pipe.predict(X_train)
accuracy_score(y_train, y_train_pred)

1.0

Teste de acurácia com base de teste

In [ ]:
from sklearn.metrics import accuracy_score

y_test_pred = pipe.predict(X_test)
accuracy_score(y_test, y_test_pred)
print("Precisão: ",accuracy_score(y_test, y_test_pred)*100,"%")

Precisão:  71.42857142857143 %


gridsearch para achar os melhores parâmetros dos componentes

In [ ]:
param_grid = {
    'reduce_dim__n_components': [1, 2, 3, 4],
    'classify__n_neighbors': [2, 3, 4, 5]
}

grid = GridSearchCV(pipe, cv=2, n_jobs=1, param_grid=param_grid, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('z-score',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('reduce_dim',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=3, random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('classify',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                             

Resultados do gridsearch

In [ ]:
grid.cv_results_['mean_test_score']

array([0.5       , 0.82142857, 0.67857143, 0.64285714, 0.5       ,
       0.82142857, 0.85714286, 0.78571429, 0.57142857, 0.82142857,
       0.85714286, 0.82142857, 0.64285714, 0.78571429, 0.85714286,
       0.71428571])

melhor resultado, com melhor configuração de parametros PCA e KNN

In [ ]:
print(grid.best_score_)
print(grid.best_params_)

0.8571428571428572
{'classify__n_neighbors': 3, 'reduce_dim__n_components': 3}


Teste de acurácia com base de teste:

In [ ]:
clf = grid.best_estimator_
y_test_pred = clf.predict(X_test)
accuracy_score(y_test, y_test_pred)
print("Precisão: ",accuracy_score(y_test, y_test_pred)*100,"%")

Precisão:  71.42857142857143 %


Utilizando StratifiedKfold, cria-se indices de treino e teste variados e  estratificados para a mesma base de dados. Para cada conjunto train/test roda-se o pipeline com gridsearch para encontrar o melhor modelo.

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=6) # objeto stratifiedKfold - n_splits seta a quantidade de conjuntos de treino/teste 
skf.get_n_splits(X, y)
print(skf)

StratifiedKFold(n_splits=6, random_state=None, shuffle=False)


In [ ]:
aux = grid # salva o grid anterior
for train_index, test_index in skf.split(X, y): # criação de indices 
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

  # print("TRAIN:", train_index, "TEST:", test_index)
  # print("X_TRAIN:", X_train, "y_TRAIN:", y_train)

  pipe.fit(X_train, y_train)
  grid = GridSearchCV(pipe, cv=2, n_jobs=1, param_grid=param_grid, scoring='accuracy')
  grid.fit(X_train, y_train)  
  print(grid.best_score_)
  print(grid.best_params_)
  clf = grid.best_estimator_
  if grid.best_score_ >= aux.best_score_: #caso encontre um score maior
    aux = grid
  y_test_pred = clf.predict(X_test)
  print("score de teste:",accuracy_score(y_test, y_test_pred))

0.7285714285714285
{'classify__n_neighbors': 2, 'reduce_dim__n_components': 2}
score de teste: 1.0
0.8309523809523809
{'classify__n_neighbors': 5, 'reduce_dim__n_components': 2}
score de teste: 0.8333333333333334
0.7952380952380952
{'classify__n_neighbors': 2, 'reduce_dim__n_components': 2}
score de teste: 0.8333333333333334
0.930952380952381
{'classify__n_neighbors': 2, 'reduce_dim__n_components': 3}
score de teste: 0.8333333333333334
0.8642857142857143
{'classify__n_neighbors': 4, 'reduce_dim__n_components': 3}
score de teste: 0.8333333333333334
0.8333333333333334
{'classify__n_neighbors': 2, 'reduce_dim__n_components': 3}
score de teste: 1.0


In [ ]:
y_test_pred = aux.predict(X_test)
accuracy_score(y_test, y_test_pred)
print("Precisão: ",accuracy_score(y_test, y_test_pred)*100,"%")

Precisão:  100.0 %
